The data set can be found from here: 
https://open.dosm.gov.my/data-catalogue/labour_labourforce_monthly_16

# 1. Import Library

Idea: by States
       

In [53]:
# Setup notebook
from pathlib import Path
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.deterministic import DeterministicProcess
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import numpy as np

# 2. Load Dataset

In [ ]:
# import data set
data_dir = Path('../datathon')

# read data set
df = pd.read_csv(data_dir / 'labourforce_monthly.csv', index_col = [0], parse_dates=[0])
df.head()

In [ ]:
# remove the empty columns
df = df.dropna(axis=1, how='all')

# Remove all the unnecessary columns only left the date and unemployed columns
df = df[['unemployed']]

# let df start from 2020
df = df.loc['2020':]

# plot the data set
fig, ax = plt.subplots(figsize=(8, 4))
plt.plot(df.index, df.unemployed, label='train')

# add title and legend to the plot
plt.title('Unemployed Rate from 2010 to 2023 in Malaysia')

display(df.head())

# 4. Split the Data set into train and test set

In [ ]:
import matplotlib.pyplot as plt
# split the data set into train and test with 80% and 20% respectively
train = df.iloc[:int(len(df)*0.8)]
test = df.iloc[int(len(df)*0.8):]

train.shape, test.shape

# 5. Linear Regression with Time Series

In [ ]:
test

In [ ]:
# make the regression plot
fig, ax = plt.subplots()
ax.plot(train.index, "unemployed", data = train, label='observed', color='#006699')
ax.set_title('Time Plot of Unemployment rate from 2010 to 2019 in Malaysia')
plt.xlabel('Date')
plt.ylabel('Unemployment Rate')
plt.show()

## 5.1 Fit the feature

### 5.1.1 Create Trend Feature

In [ ]:
moving_average = df.unemployed.rolling(
    window=12,       # 12-months window
    center=True,      # puts the average at the center of the window
    min_periods=6,  # choose about half the window size
).mean()              # compute the mean (could also do median, std, min, max, ...)

ax = df.unemployed.plot(style=".", color="0.5")
moving_average.plot(
    ax=ax, linewidth=3, title="Unemployment rate", legend=False,
)

In [ ]:
train['count'] = range(1, len(train) + 1)

X = train[['count']]  # features
y = train['unemployed']  # target

# create the trend feature
dp = DeterministicProcess(
    index=train.index,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=2,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
)

X = dp.in_sample()
X

### 5.1.2 Create Seasonality

In [ ]:
# Assuming you have the 'train' DataFrame with a date column
# Convert the date column to a DateTimeIndex
#train['date'] = pd.to_datetime(train['date'])
#train.index = train['date']

# Calculate the quarter for each timestamp
train['quarter'] = (train.index.month - 1) // 3 + 1

# Create quarterly seasonality indicator variables
train['quarter_Q1'] = (train['quarter'] == 1).astype(int)
train['quarter_Q2'] = (train['quarter'] == 2).astype(int)
train['quarter_Q3'] = (train['quarter'] == 3).astype(int)
train['quarter_Q4'] = (train['quarter'] == 4).astype(int)

# Create the feature matrix (X) including quarterly seasonality indicators
X1 = train[['quarter_Q1', 'quarter_Q2', 'quarter_Q3', 'quarter_Q4']]
X1

# remove the quarter column in train
train = train.drop(['quarter_Q1'], axis=1)
train = train.drop(['quarter_Q2'], axis=1)
train = train.drop(['quarter_Q3'], axis=1)
train = train.drop(['quarter_Q4'], axis=1)

In [ ]:
#train['date'] = train.index
X['date'] = train.index
#X['date']  = X.index
X.index = pd.to_datetime(X.index)

# Create Seosonal Feature
fourier = CalendarFourier(freq="M", order=12)  # 10 sin/cos pairs for "A"nnual seasonality

dp = DeterministicProcess(
    index= train.index,
    constant=True,               # dummy feature for bias (y-intercept)
    order=3,                     # trend (order 1 means linear)
    seasonal=True,               # weekly seasonality (indicators)
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True                  # drop terms to avoid collinearity
)

X = dp.in_sample()  # create features for dates in tunnel.index


In [ ]:
train.index = pd.to_datetime(train.index, unit='s')
train.index

## 5.2 Fit the Model

In [ ]:
X.head()

In [ ]:
len(test.index)

In [ ]:
model = LinearRegression(fit_intercept=False)
_ = model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=y.index)
X_fore = dp.out_of_sample(steps=9, forecast_index=test.index)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(color='0.25', style='.', title="Unemployment Rate")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax = y_fore.plot(ax=ax, label="Seasonal Forecast", color='C3')
_ = ax.legend()
test['unemployed'].plot(legend=True, figsize=(12,8))

In [ ]:
ax = y.plot(alpha=0.5)
ax = y_pred.plot(ax=ax, linewidth=3)
ax.set_title('Time Plot of Unemployment rate from 2010 to 2023 in Malaysia')

In [57]:
rms = sqrt(mean_squared_error(test.unemployed,y_fore))
print("RMSE: ", rms)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test.unemployed, y_fore)
print("MAE: ", mae)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(test.unemployed, y_fore)
print("MAPE: ", mape)

mse = mean_squared_error(test.unemployed, y_fore)
print(f"MSE: {mse:.2f}")

RMSE:  114.77974461357205
MAE:  89.20493324763414
MAPE:  15.26653360828816
MSE: 13174.39


In [51]:
test

,unemployed
date,
2022-11-01,600.9
2022-12-01,599.6
2023-01-01,596.1
2023-02-01,591.9
2023-03-01,588.7
2023-04-01,586.9
2023-05-01,584.6
2023-06-01,581.7
2023-07-01,579.2


## 5.3 Hybrid Forecasting with Residuals

In [ ]:
# 1. Train and predict with first model
model_1.fit(X_train_1, y_train)
y_pred_1 = model_1.predict(X_train)

# 2. Train and predict with second model on residuals
model_2.fit(X_train_2, y_train - y_pred_1)
y_pred_2 = model_2.predict(X_train_2)

# 3. Add to get overall predictions
y_pred = y_pred_1 + y_pred_2